In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install openpyxl

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import geopandas
import matplotlib.pyplot as plt
import cartopy.crs as crs
import cartopy.feature as cfeature

In [4]:
airport_df = pd.read_csv("/kaggle/input/dav-assignment/Airports.csv", encoding="unicode_escape")
flights_df = pd.read_excel("/kaggle/input/dav-assignment/Flight Data.xlsx",sheet_name = '2003-2009')
citiy = pd.read_csv("/kaggle/input/world-city/worldcities.csv")

In [5]:
flights_df.head()

In [6]:
flights_df= flights_df[flights_df['TimeSeries']=='2003-07']

In [7]:
#df['TimeSeries'].unique()

In [8]:
airport_df.columns = ['Source', 'label', 'country', 'lat', 'lon']

In [9]:
airport_df = airport_df[airport_df['lon'] != -70]

# Networks

In [10]:
def Airport_network_grhps(Contry1,Contry2,flights_df,airport_df,N):
    airport_df.columns = ['Source', 'label', 'country', 'lat', 'lon']
    routes = flights_df[flights_df['Target Country']==flights_df['Target Country']]# only consider indoorflights
    routes = routes[routes['Source Country']==Contry1]
    airport_df = airport_df[airport_df["country"]==Contry2]
    routes = routes.merge(airport_df , how='inner',on='Source')
    list_source = routes['Source City'].unique()
    airports = routes[routes["Target City"].isin(list_source)]
    airports = airports[airports['Source Country']==airports['Target Country']]
    edges = airports[['Source City', 'Target City']].values
    
    g = nx.from_pandas_edgelist(airports,source='Source City',target="Target City",edge_attr="Weight")
    
    pos = {v['Source City']: (v['lon'], v['lat'])
       for airport, v in
       airports.to_dict('index').items()}
    
    
    deg = nx.degree(g)
    sizes = [5 * deg[iata] for iata in g.nodes]
    
    labels = {iata: iata if deg[iata] >= N else ''
          for iata in g.nodes}
    
    
    fig, ax = plt.subplots(
    1, 1, figsize=(24, 24),
    subplot_kw=dict(projection=crs.PlateCarree()))
    ax.set_title('Network Grhps of ' +Contry1 )
    ax.coastlines()
    nx.draw_networkx(g, ax=ax,
                 font_size=16,
                 alpha=.5,
                 width=.075,
                 node_size=sizes,
                 pos=pos,
                 cmap=plt.cm.autumn, labels=labels)
    return g

**USA Network**

In [11]:
Airport_network_grhps('USA','United States',flights_df,airport_df,100)

In [12]:
Airport_network_grhps("China", "China",flights_df,airport_df,50)

In [13]:
Airport_network_grhps("United Kingdom", "United Kingdom",flights_df,airport_df,10)

In [14]:
Airport_network_grhps("Australia", "Australia",flights_df,airport_df,20)

# Degree distribution

In [15]:
def Weight_Degree_distribution(Contry1,Contry2,flights_df,airport_df):
    routes = flights_df[flights_df['Target Country']==flights_df['Target Country']] # only consider indoorflights
    routes = routes[routes['Source Country']==Contry1]
    
    airport_df = airport_df[airport_df["country"]==Contry2]
    routes = routes.merge(airport_df , how='inner',on='Source')
    
    list_source = routes['Source City'].unique()
    
    airports = routes[routes["Target City"].isin(list_source)]
    
    airports = airports[airports['Source Country']==airports['Target Country']]
    
    g = nx.from_pandas_edgelist(airports,source='Source City',target="Target City",edge_attr="Weight",create_using=nx.DiGraph())
    
    degree_sequence = sorted([d for n, d in g.degree(g, weight="Weight")], reverse=True)
    fig, ax = plt.subplots(
    1, 1, figsize=(12, 8)
    )
    ax.plot(degree_sequence, marker="o")
    ax.set_title('Degree distribution' + " of "+ Contry1+" 2003-07",fontsize=18)
    ax.set_ylabel("Degree")
    ax.set_xlabel("Rank")
    ax.set_yscale("log")
    plt.show()

**USA Degree distribution**

In [16]:
Weight_Degree_distribution('USA','United States',flights_df,airport_df)

# Degree vs. betweenness

In [17]:
def Weighted_degree_betweenness(Contry1,Contry2,flights_df,airport_df):
    routes = flights_df[flights_df['Target Country']==flights_df['Target Country']] # only consider indoorflights
    routes = routes[routes['Source Country']==Contry1]
    
    airport_df = airport_df[airport_df["country"]==Contry2]
    routes = routes.merge(airport_df , how='inner',on='Source')
    
    list_source = routes['Source City'].unique()
    
    airports = routes[routes["Target City"].isin(list_source)]
    
    airports = airports[airports['Source Country']==airports['Target Country']]
    
    g = nx.from_pandas_edgelist(airports,'Source City', 'Target City',edge_attr='Weight',create_using=nx.DiGraph())
    degree_sequence = nx.degree(g)
    betweenness = nx.betweenness_centrality(g,weight="weight", normalized=False)
    x = [degree_sequence[iata] for iata in g.nodes]
    y = [betweenness[iata] for iata in g.nodes]
    

    fig = plt.figure(figsize=(18, 10))
    ax = fig.add_subplot(1, 1, 1)
    plt.scatter(x, y, alpha=0.75)

    plt.title(r'Degree vs Betweenness' + " for "+ Contry1 + " 2003-07",fontsize=18)
    plt.ylabel(r"Betweenness")
    plt.yscale("log")
    plt.xscale("log")
    plt.xlabel(r"Weighted Degree")
    plt.show()

In [18]:
Weighted_degree_betweenness('USA','United States',flights_df,airport_df)

In [19]:
Weighted_degree_betweenness("China", "China",flights_df,airport_df)

In [20]:
Weighted_degree_betweenness("United Kingdom", "United Kingdom",flights_df,airport_df)

In [21]:
Weighted_degree_betweenness("Australia", "Australia",flights_df,airport_df)

# 

# Assortativity

In [22]:
def Assortativity(Contry1,Contry2,flights_df,airport_df):
    routes = flights_df[flights_df['Target Country']==flights_df['Target Country']] # only consider indoorflights
    routes = routes[routes['Source Country']==Contry1]
    
    airport_df = airport_df[airport_df["country"]==Contry2]
    routes = routes.merge(airport_df , how='inner',on='Source')
    
    list_source = routes['Source City'].unique()
    
    airports = routes[routes["Target City"].isin(list_source)]
    
    airports = airports[airports['Source Country']==airports['Target Country']]
    
    g = nx.from_pandas_edgelist(airports,'Source City', 'Target City',create_using=nx.DiGraph())
    
    r = nx.degree_assortativity_coefficient(g,weight="weight")
    Aeso = nx.degree_pearson_correlation_coefficient(g,x='out',y='in')
    
    return r

In [23]:
Assortativity('USA','United States',flights_df,airport_df)

In [24]:
Assortativity("China", "China",flights_df,airport_df)

In [25]:
Assortativity("United Kingdom", "United Kingdom",flights_df,airport_df)

In [26]:
Assortativity("Australia", "Australia",flights_df,airport_df)

In [27]:
def Degree_Corrlation(Contry1,Contry2,flights_df,airport_df):
    routes = flights_df[flights_df['Target Country']==flights_df['Target Country']] # only consider indoorflights
    routes = routes[routes['Source Country']==Contry1]
    
    airport_df = airport_df[airport_df["country"]==Contry2]
    routes = routes.merge(airport_df , how='inner',on='Source')
    
    list_source = routes['Source City'].unique()
    
    airports = routes[routes["Target City"].isin(list_source)]
    
    airports = airports[airports['Source Country']==airports['Target Country']]
    
    g = nx.from_pandas_edgelist(airports,'Source City', 'Target City',edge_attr="Weight",create_using=nx.DiGraph())
    degree_sequence = [(n,d) for n, d in nx.degree(g)]
    degree_sequence = sorted(degree_sequence)
    
    
    ave_n_d = sorted((nx.average_neighbor_degree(g,source='out',target='in').items()))
    
    
    degree_sequence_df = pd.DataFrame(degree_sequence,columns=['node','deg'])
    ave_n_d_df = pd.DataFrame(ave_n_d,columns=['node','neig'])
    
    
    fig = plt.figure(figsize=(18, 10))
    ax = fig.add_subplot(1, 1, 1)
    plt.scatter(degree_sequence_df['deg'],ave_n_d_df['neig'] , alpha=0.75)

    plt.title(r'Assortativity plot' + " for "+ Contry1 + " 2003-07" ,fontsize=20)
    plt.ylabel(r"Average Neighbour Degree(log scale)")
    plt.yscale("log")
    plt.xscale("log")
    plt.xlabel(r"Node Degree(log scale)")
    plt.show()

In [28]:
Degree_Corrlation("USA", "United States",flights_df,airport_df)

In [29]:
Degree_Corrlation("China", "China",flights_df,airport_df)

In [30]:
Degree_Corrlation("United Kingdom", "United Kingdom",flights_df,airport_df)

In [31]:
Degree_Corrlation("Australia", "Australia",flights_df,airport_df)

# Core Clusters

In [32]:
def Airport_network_grhps_core(Contry1,Contry2,flights_df,airport_df,N):
    airport_df.columns = ['Source', 'label', 'country', 'lat', 'lon']
    routes = flights_df[flights_df['Target Country']==flights_df['Target Country']]# only consider indoorflights
    routes = flights_df[flights_df['TimeSeries']=='2003-07']
    routes = routes[routes['Source Country']==Contry1]
    
    airport_df = airport_df[airport_df["country"]==Contry2]
    routes = routes.merge(airport_df , how='inner',on='Source')
    
    list_source = routes['Source City'].unique()
    
    airports = routes[routes["Target City"].isin(list_source)]
    
    airports = airports[airports['Source Country']==airports['Target Country']]
    edges = airports[['Source City', 'Target City']].values
    
    g = nx.from_pandas_edgelist(airports,source='Source City',target="Target City",edge_attr="Weight",create_using=nx.DiGraph())
    g.remove_edges_from(list(nx.selfloop_edges(g)))
    core = nx.k_core(g, core_number=nx.core_number(g))

    color_map = []
    for node in g:
        if node in core:
            color_map.append('blue')
        else: 
            color_map.append('green')
    
    pos = {v['Source City']: (v['lon'], v['lat'])
       for airport, v in
       airports.to_dict('index').items()}
    
    
    deg = nx.degree(g)
    sizes = [5 * deg[iata] for iata in g.nodes]
    
    labels = {iata: iata if deg[iata] >= N else ''
          for iata in g.nodes}
    
    
    fig, ax = plt.subplots(
    1, 1, figsize=(24, 24),
    subplot_kw=dict(projection=crs.PlateCarree()))
    ax.coastlines()
    ax.
    nx.draw_networkx(g, ax=ax,
                 font_size=16,
                 alpha=.5,
                 width=.075,
                 node_size=sizes,
                 pos=pos,
                 cmap=plt.cm.autumn, labels=labels, node_color=color_map)
    return g

In [33]:
Airport_network_grhps_core("USA", "United States",flights_df,airport_df,150)

In [34]:
Airport_network_grhps_core("China", "China",flights_df,airport_df,40)

In [35]:
Airport_network_grhps_core("United Kingdom", "United Kingdom",flights_df,airport_df,10)

In [36]:
Airport_network_grhps_core("Australia", "Australia",flights_df,airport_df,30)

# Core_Periphery_Structure

In [37]:
def Core_Periphery_Structure(Contry1,Contry2,flights_df,airport_df):
    routes = flights_df[flights_df['Target Country']==flights_df['Target Country']] # only consider indoorflights
    routes = routes[routes['Source Country']==Contry1]
    
    airport_df = airport_df[airport_df["country"]==Contry2]
    routes = routes.merge(airport_df , how='inner',on='Source')
    
    list_source = routes['Source City'].unique()
    
    airports = routes[routes["Target City"].isin(list_source)]
    
    airports = airports[airports['Source Country']==airports['Target Country']]
    
    g = nx.from_pandas_edgelist(airports,'Source City', 'Target City',edge_attr='Weight',create_using=nx.DiGraph())
    
    
    g.remove_edges_from(list(nx.selfloop_edges(g)))
    core = nx.k_core(g, core_number=nx.core_number(g))
    s=(len(core))
    
    df_deg = pd.DataFrame([(n, v) for (n, v) in sorted(g.degree, key = lambda x: x[1], reverse = True)]).rename(columns = {0:'id', 1:'degree'}, inplace = False)
    df_deg['rank'] = df_deg['degree'].rank(ascending = False)
    hi_co_deg = []
    for id in df_deg['id']:
        count = 0
        for i in g.neighbors(id):
            if df_deg[df_deg['id'] == i].iloc[0]['degree'] > df_deg[df_deg['id'] == id].iloc[0]['degree']:
                count += 1
        hi_co_deg.append(count)
        
    x = pd.DataFrame([v for (n, v) in sorted(g.degree, key = lambda x: x[1], reverse = False)]).rank(axis = 0, method = 'first').to_numpy()
    y = hi_co_deg
    
    fig = plt.figure(figsize=(18, 10))
    ax = fig.add_subplot(1, 1, 1)
    plt.plot(x, y)
    plt.title('Core Periphery Structure '+Contry1 +" 2003-07" ,fontsize=20)
    plt.xlabel('Node Number - Ranked')
    plt.ylabel('No of connection to higher Degree Nodes')
    plt.xscale('linear')
    plt.yscale('linear')
    plt.legend(['Core Community Size ' +str(s)])
    plt.show()

In [38]:
Core_Periphery_Structure("USA", "United States",flights_df,airport_df)

In [39]:
Core_Periphery_Structure("China", "China",flights_df,airport_df)

In [40]:
Core_Periphery_Structure("United Kingdom", "United Kingdom",flights_df,airport_df)

In [41]:
Core_Periphery_Structure("Australia", "Australia",flights_df,airport_df)

In [42]:
def Assortativity_hist(Contry1,Contry2,flights_df,airport_df):
    routes = flights_df[flights_df['Target Country']==flights_df['Target Country']] # only consider indoorflights
    routes = routes[routes['Source Country']==Contry1]
    
    airport_df = airport_df[airport_df["country"]==Contry2]
    routes = routes.merge(airport_df , how='inner',on='Source')
    
    list_source = routes['Source City'].unique()
    
    airports = routes[routes["Target City"].isin(list_source)]
    
    airports = airports[airports['Source Country']==airports['Target Country']]
    
    g = nx.from_pandas_edgelist(airports,'Source City', 'Target City',edge_attr="Weight",create_using=nx.DiGraph())
    val = nx.average_degree_connectivity(g, weight="weight",source='in',target='out')
    #df= pd.DataFrame.from_dict(val)
    #df.columns = ['node','Neighbours']
    fig = plt.figure(figsize=(18, 10))
    ax = fig.add_subplot(1, 1, 1)
    plt.scatter(val.keys(),val.values(), alpha=0.75)

    plt.title(r'Degree vs Betweenness' + " for "+ Contry1)
    plt.ylabel(r"Betweenness")
    plt.yscale("log")
    plt.xscale("log")
    plt.xlabel(r"Degree")
    plt.show()
    return val

In [43]:
routes = flights_df[flights_df['Target Country']==flights_df['Target Country']] # only consider indoorflights
routes = routes[routes['Source Country']==Contry1]
    
airport_df = airport_df[airport_df["country"]==Contry2]
routes = routes.merge(airport_df , how='inner',on='Source')
    
list_source = routes['Source City'].unique()
    
airports = routes[routes["Target City"].isin(list_source)]
    
airports = airports[airports['Source Country']==airports['Target Country']]
    
g = nx.from_pandas_edgelist(airports,'Source City', 'Target City',edge_attr="Weight",create_using=nx.DiGraph())
val = nx.average_degree_connectivity(g, weight="weight")

In [ ]:
Assortativity_hist("USA", "United States",flights_df,airport_df)